In [ ]:
!pip install timm
!pip install grad-cam

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2


from PIL import Image
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt

#import gradcam library
from pytorch_grad_cam import GradCAM, ScoreCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
class Config:
    seed = 42
    epochs = 10
    train_img_dir = "../input/petfinder-pawpularity-score/train"
    image_size = 384
    n_splits = 10
    model_name = 'swin_large_patch4_window12_384'
    train_batchsize = 64
    val_batchsize = 64
    debug = False
    
    fp16 = True
    
    #optimizer
    optimizer = "Adam"
    
    if optimizer == "Adam":
        optimizer_params = dict(
            lr = 1e-3,
            betas = (0.9, 0.999),
            eps = 1e-8,
            weight_decay = 0,
            amsgrad = False
        )
        
    elif optimizer == "SGD":
        optimizer_params = dict(
            lr = 1e-3,
            weight_decay = 0,
            dampening  = 0,
            nesterov = False
        ) 
        
    # Scheduler
    scheduler = 'CosineAnnealingLR' # CosineAnnealingLR, ReduceLROnPlateau
    if scheduler == "CosineAnnealingLR":
        scheduler_params = dict(
            T_max = epochs,
            eta_min = 0,
            last_epoch = -1,
            verbose = False
        )
    elif scheduler == "ReduceLROnPlateau":
        scheduler_params = dict(
            mode = "min",
            factor= 0.1,
            patience=4,
            threshold=1e-4,
            min_lr = 1e-6
        )

In [ ]:
class PawModel(nn.Module):
    def __init__(self, model_name):
        super(PawModel, self).__init__()
        self.backbone = timm.create_model(model_name, pretrained=True, in_chans=3) #this will create a model with classifier
        self.backbone.head = nn.Linear(self.backbone.head.in_features, 128)
#         self.backbone.classifier = nn.Identity()
        self.dropout = nn.Dropout(p=0.1)
        self.fcn = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        
    def forward(self,image_array):
        x = self.backbone(image_array)
        x = self.dropout(x)
        x = self.fcn(x)
        return self.output(x)


In [ ]:
transform = A.Compose([
    A.Resize(Config.image_size, Config.image_size,p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1),
    ToTensorV2()
], p=1)

## Loading model

In [ ]:
trained_model = PawModel(Config.model_name)
trained_model.load_state_dict(torch.load("../input/pawpularity-swin-models/swin_large_patch4_window12_384_n_fold_0_best.pth",map_location=torch.device('cpu')))

## Create gradcam wrapper

In [ ]:
def reshape_transform(tensor, height=12, width=12):
    """
    Following the guide from https://github.com/jacobgil/pytorch-grad-cam
    """
    result = tensor.reshape(tensor.size(0),
        height, width, tensor.size(2))
    # Bring the channels to the first dimension,
    # like in CNNs.
    result = result.transpose(2, 3).transpose(1, 2)
    return result

target_layers = [trained_model.backbone.layers[-1].blocks[-1].norm1]
grad_cam = GradCAM(model=trained_model, target_layers=target_layers, reshape_transform=reshape_transform, use_cuda=False)

## Helper function to get gradcam visualization

In [ ]:
def get_visualization(grad_cam, image_fp, transform):
    image = Image.open(image_fp)
    original_image = image.copy().resize((Config.image_size,Config.image_size))
    original_image = np.array(original_image, dtype=np.float32)/255.0
    input_image = transform(image=np.array(image, dtype=np.float32))['image']
    output = grad_cam(input_tensor = input_image.unsqueeze(0))
    return original_image, show_cam_on_image(original_image, output[0], use_rgb=True)
                      

## Randomly selecting 10 images for showing

In [ ]:
train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
train_df = train_df.sample(10)
train_df['filepath'] = train_df["Id"].map(lambda x: os.path.join("../input/petfinder-pawpularity-score/train", x+".jpg"))
train_df.head()

In [ ]:
image_fps = train_df['filepath'].tolist()
fig = plt.figure(figsize=(20,80))
for i, image_fp in enumerate(image_fps):
    original_ax = fig.add_subplot(10,2,1+i*2)
    gradcam_ax = fig.add_subplot(10,2,2+i*2)
    
    original_image, grad_cam_image = get_visualization(grad_cam,image_fp,transform)
    original_ax.imshow(original_image)
    gradcam_ax.imshow(grad_cam_image)
    
    original_ax.axis("off")
    original_ax.set_title("Original")
    gradcam_ax.axis("off")
    gradcam_ax.set_title("Gradcam")

In [ ]:
fig.savefig("swinTransformer_gradcam.png")